In [2]:
import pandas as pd
import numpy as np

# Loading the Data


In [27]:
bs = pd.read_json("branch_service_transaction_info.json")
ct = pd.read_json("customer_transaction_info.json")

# Profiling the Data

In [9]:
bs.head(10) # view the first x records of the DataFrame

,txn_id,branch_name,service,price
0,TXN-24546,MallOfAsia,Manicure,NaN
1,TXN-14642,Starmall,HairColor,NaN
2,TXN-60295,SmallMall,FootSpa,NaN
3,TXN-60295,Starmall,FootSpa,NaN
4,TXN-60295,MayMall,FootSpa,NaN
5,TXN-60295,FrankMall,FootSpa,NaN
6,TXN-40462,Starmall,HairColor,NaN
7,TXN-40462,Megamall,HairColor,NaN
8,TXN-08102,RobinsonsMall,HairColor,NaN
9,TXN-08102,SmallMall,HairColor,NaN


In [10]:
bs.tail(10)

,txn_id,branch_name,service,price
130643,TXN-11897,FrankMall,FootSpa,100.121230
130644,TXN-11897,RobinsonsMall,FootSpa,100.121230
130645,TXN-11897,FrankMall,FootSpa,100.121230
130646,TXN-43876,,Haircut,66.123457
130647,TXN-65468,RobinsonsMall,Haircut,66.123457
130648,TXN-65468,MallOfAsia,Haircut,66.123457
130649,TXN-60822,MallOfAsia,Rebond,400.231230
130650,TXN-60822,FrankMall,Rebond,400.231230
130651,TXN-01784,RobinsonsMall,HairColor,88.093930
130652,TXN-01784,MallOfAsia,HairColor,88.093930


In [11]:
print(bs.shape)
print(ct.shape)

(130653, 4)
(130653, 5)


In [15]:
# get the number of unique items in the column
print(bs['txn_id'].nunique())
print(ct['txn_id'].nunique())

62354
62354


In [14]:
bs['service'].unique() # get the unique values in that column 

array(['Manicure', 'HairColor', 'FootSpa', 'Rebond', 'Haircut',
       'NailColor', 'Pedicure'], dtype=object)

In [17]:
bs['price'].describe()

count    110653.000000
mean         95.966969
std         115.874117
min           0.000000
25%          30.123790
50%          66.123457
75%          88.093930
max         400.231230
Name: price, dtype: float64

In [22]:
print(ct['avail_date'].min())
print(ct['avail_date'].max())

2005-01-01
2030-12-30


In [24]:
ct['last_name'].iloc[50000:70000] # get certain values in this column

50000               Heaney
50001               Heaney
50002            Marquardt
50003            Marquardt
50004            Marquardt
               ...        
69995        Weber,,,,,,,,
69996      Cormier,,,,,,,,
69997       Jacobs,,,,,,,,
69998      Murazik,,,,,,,,
69999         Kuhn,,,,,,,,
Name: last_name, Length: 20000, dtype: object

# Creating the Data Pipeline Diagram

## Data Cleaning

In [28]:
# drop the duplicates
print(ct.shape)
ct = ct.drop_duplicates()
print(ct.shape)

(130653, 5)
(72355, 5)


In [29]:
ct.to_parquet('customer_txn_duplicates_removed.parquet')

In [30]:
ct['last_name'] = ct['last_name'].str.lower()
ct.head(2)

,txn_id,avail_date,last_name,first_name,birthday
0,TXN-24546,2030-09-08,ortiz,EDUARDO,1990-07-08
1,TXN-14642,2026-05-26,nienow,LEA,2000-11-26


In [31]:
ct.to_parquet('customer_txn_last_name_clean.parquet')

## Yearly View per Branch

In [32]:
merged = pd.merge(ct, bs)
merged.to_parquet('merged_data.parquet')

In [33]:
merged

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
0,TXN-24546,2030-09-08,ortiz,EDUARDO,1990-07-08,MallOfAsia,Manicure,NaN
1,TXN-24546,2030-09-08,ortiz,EDUARDO,1990-07-08,MallOfAsia,Manicure,55.232400
2,TXN-24546,2012-04-03,ortiz,Eduardo,1999-04-08,MallOfAsia,Manicure,NaN
3,TXN-24546,2012-04-03,ortiz,Eduardo,1999-04-08,MallOfAsia,Manicure,55.232400
4,TXN-14642,2026-05-26,nienow,LEA,2000-11-26,Starmall,HairColor,NaN
...,...,...,...,...,...,...,...,...
166817,TXN-43076,2021-02-27,champlin,Queen,1995-03-29,MayMall,Haircut,66.123457
166818,TXN-43076,2021-02-27,champlin,Queen,1995-03-29,MayMall,Haircut,66.123457
166819,TXN-43065,2025-01-05,wyman,Giovanny,1993-01-10,Megamall,Rebond,400.231230
166820,TXN-43065,2025-01-05,wyman,Giovanny,1993-01-10,MallOfAsia,Rebond,400.231230


In [35]:
merged['avail_date'] = pd.to_datetime(merged['avail_date'])
merged.groupby([merged.avail_date.dt.year, 'branch_name'])['price'].sum().to_frame() # query by a group by similar to sql

price
avail_date branch_name                
2005                       4298.024691
           FrankMall      56711.249909
           MallOfAsia     62502.141612
           MayMall        65314.419050
           Megamall       63722.520395
...                                ...
2030       Megamall       61514.464025
           N/A             2048.417700
           RobinsonsMall  56816.266169
           SmallMall      60680.995068
           Starmall       64301.362128

[234 rows x 1 columns]

We then separate these jobs into separate python files for scheduling. We'll be using chron for scheduling